# kernel

> IPythonKernel based on pystata

The latest documentation for implementing a wrapper kernel is [here](https://jupyter-client.readthedocs.io/en/latest/wrapperkernels.html), but the current code deviates from those instructions (which call for inheriting from [kernelbase.Kernel](https://github.com/ipython/ipykernel/blob/main/ipykernel/kernelbase.py)), instead inheriting from the IPython kernel implementation, [IPythonKernel](https://github.com/ipython/ipykernel/blob/main/ipykernel/ipkernel.py).

The following diagram shows the main dependencies among the principal `nbstata` modules:

```{mermaid}
flowchart TB
  A[kernel] -.-> I[config]
  A -.-> B[cell]
  B -.-> E[magics]
  B -.-> D[stata_session]
  D -.-> C[noecho]
  E -.-> L[browse]
  A -.-> F[completions]
  F -.-> D
  F -.-> G[completion_env]
  A -.-> N[inspect]
  N -.-> H[stata_more]
  D -.-> H
  L -.-> M[pandas]
  L -.-> H
  M -.-> H
  C -.-> H
```

In [ ]:
#| default_exp kernel
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from nbstata.config import get_config, launch_stata
from nbstata.misc_utils import print_red
from nbstata.inspect import get_inspect
from nbstata.stata_session import StataSession
from nbstata.completions import CompletionsManager
from nbstata.cell import Cell
import nbstata # for __version__
from fastcore.basics import patch_to
from ipykernel.ipkernel import IPythonKernel

In [ ]:
#| export
class PyStataKernel(IPythonKernel):
    """A jupyter kernel based on pystata"""
    implementation = 'nbstata'
    implementation_version = nbstata.__version__
    language_info = {
        'name': 'stata',
        'version': '17',
        'mimetype': 'text/x-stata',
        'file_extension': '.do',
    }
    banner = "nbstata: a Jupyter kernel for Stata based on pystata"
    help_links = [
        {
            "text": "Stata Documentation",
            "url": "https://www.stata.com/features/documentation/",
        },
        {
            "text": "nbstata Help",
            "url": "https://hugetim.github.io/nbstata/",
        },
    ]

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.stata_ready = False
        self.shell.execution_count = 0
        self.sc_delimiter = False
        self.perspective_enabled = None
        self.inspect_output = "Stata not yet initialized."
        try:
            self.init_stata()
        except ModuleNotFoundError as err:
            pass # wait for first do_execute so error message can be displayed under cell

In [ ]:
#| export
def _config_stata(env):
    launch_stata(env['stata_dir'], 
                 env['edition'],
                 False if env['splash']=='False' else True,
                 env['graph_format'],
                )

In [ ]:
#| export
@patch_to(PyStataKernel)
def init_stata(self):
    self.env = get_config()
    _config_stata(self.env)
    self.stata_session = StataSession()
    self.completions = CompletionsManager(self.stata_session)
    self.inspect_output = ""
    self.stata_ready = True

In [ ]:
#| export
_missing_stata_message = (
    "pystata path not found\n"
    "A Stata 17 installation is required to use the nbstata Stata kernel. "
    "If you already have Stata 17 installed, "
    "please specify its path in your configuration file."
)

In [ ]:
#| hide
print(_missing_stata_message)

pystata path not found
A Stata 17 installation is required to use the nbstata Stata kernel. If you already have Stata 17 installed, please specify its path in your configuration file.


In [ ]:
#| export
def _handle_stata_import_error(err, silent, execution_count):
    if not silent:
        print_red(f"ModuleNotFoundError: {_missing_stata_message}")
    return {
        "traceback": [],
        "ename": "ModuleNotFoundError",
        "evalue": _missing_stata_message,
        'status': "error",
        'execution_count': execution_count,
    }

In [ ]:
#| export
def print_stata_error(text):
    lines = text.splitlines()
    if len(lines) > 2:
        print("\n".join(lines[:-2]))
    print_red("\n".join(lines[-2:]))

In [ ]:
from textwrap import dedent

In [ ]:
print_stata_error(dedent("""\
    output prior to error
    error message
    error code
    """))

output prior to error
error message
error code


In [ ]:
#| export
def _handle_stata_error(err, silent, execution_count):
    reply_content = {
        "traceback": [],
        "ename": "Stata error",
        "evalue": str(err),
    }
    if not silent:
        print_stata_error(reply_content['evalue'])
#         self.send_response(
#             self.iopub_socket,
#             "error",
#             reply_content,
#         )
    reply_content.update({
        'status': "error",
        'execution_count': execution_count,
    })
    return reply_content

In [ ]:
#| export
@patch_to(PyStataKernel)
def post_do_hook(self):
    self.inspect_output = ""

In [ ]:
#| export
@patch_to(PyStataKernel)
def do_execute(self, code, silent,
               store_history=True, user_expressions=None, allow_stdin=False):
    """Execute Stata code cell"""
    if not self.stata_ready:
        try:
            self.init_stata()
        except ModuleNotFoundError as err:
            return _handle_stata_import_error(err, silent, self.execution_count)
    self.shell.execution_count += 1
    code_cell = Cell(self, code, silent)
    try:
        code_cell.run()
    except SystemError as err:
        return _handle_stata_error(err, silent, self.execution_count)
    self.sc_delimiter = code_cell.sc_delimiter
    self.post_do_hook()
    return {
        'status': "ok",
        'execution_count': self.execution_count,
        'payload': [],
        'user_expressions': {},
    }

In [ ]:
#| export
@patch_to(PyStataKernel)
def do_complete(self, code, cursor_pos):
    """Provide context-aware suggestions"""
    if self.stata_ready:
        cursor_start, cursor_end, matches = self.completions.do(
            code,
            cursor_pos,
            self.sc_delimiter,
        )
    else:
        cursor_start = cursor_end = cursor_pos
        matches = []
    return {
        'status': "ok",
        'cursor_start': cursor_start,
        'cursor_end': cursor_end,
        'metadata': {},
        'matches': matches,
    }

In [ ]:
#| export
@patch_to(PyStataKernel)
def do_is_complete(self, code):
    """Overrides IPythonKernel with kernelbase default"""
    return {"status": "unknown"}

In [ ]:
#| export
@patch_to(PyStataKernel)
def do_inspect(self, code, cursor_pos, detail_level=0, omit_sections=()):
    """Display Stata 'describe' output regardless of cursor position"""
    if not self.inspect_output:
        self.inspect_output = get_inspect(code, cursor_pos, detail_level, omit_sections)
    data = {'text/plain': self.inspect_output}
    return {"status": "ok", "data": data, "metadata": {}, "found": True}

In [ ]:
#| export
@patch_to(PyStataKernel)
def do_history(
    self,
    hist_access_type,
    output,
    raw,
    session=None,
    start=None,
    stop=None,
    n=None,
    pattern=None,
    unique=False,
):
    """Overrides IPythonKernel with kernelbase default"""
    return {"status": "ok", "history": []}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()